# Import Required Libraries


In [ ]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model


# Load Saved Files

In [ ]:
# Load trained model
model = load_model("autism_model.h5")

# Load scaler and encodings
scaler = joblib.load("scaler.pkl")
t_int_encodings = joblib.load("t_int_encodings.pkl")


# Prepare a New Input Sample

In [ ]:
# Prepare a New Input Sample
new_sample = {
    'A1_Score': 1,
    'A2_Score': 1,
    'A3_Score': 1,
    'A4_Score': 1,
    'A5_Score': 1,
    'A6_Score': 1,
    'A7_Score': 1,
    'A8_Score': 1,
    'A9_Score': 1,
    'A10_Score': 1,
    'age': 5,
    'gender': 'male',
    'ethnicity': 'White-European',
    'jundice': 'yes',
    'contry_of_res': 'United States',
    'used_app_before': 'no',
    'result': 10,
    'relation': 'Parent'
}


# Apply T-Int Encoding to Categorical Columns

In [ ]:
# Convert to DataFrame
new_df = pd.DataFrame([new_sample])

categorical_cols = ['gender', 'ethnicity', 'jundice', 'contry_of_res', 'used_app_before', 'relation']

# Strip spaces and apply T-Int encoding
for col in categorical_cols:
    new_df[col] = new_df[col].astype(str).str.strip()
    new_df[col] = new_df[col].map(t_int_encodings[col])


In [ ]:
print("Check for NaNs after encoding:\n", new_df.isna().sum())


Check for NaNs after encoding:
 A1_Score           0
A2_Score           0
A3_Score           0
A4_Score           0
A5_Score           0
A6_Score           0
A7_Score           0
A8_Score           0
A9_Score           0
A10_Score          0
age                0
gender             1
ethnicity          0
jundice            0
contry_of_res      0
used_app_before    0
result             0
relation           0
dtype: int64


In [ ]:
for col in categorical_cols:
    new_df[col] = new_df[col].astype(str).str.strip()
    new_df[col] = new_df[col].map(t_int_encodings[col])
    # Handle unseen categories
    if new_df[col].isna().any():
        print(f"Unseen value in column: {col}, filling with mean.")
        new_df[col].fillna(t_int_encodings[col].mean(), inplace=True)


Unseen value in column: gender, filling with mean.
Unseen value in column: ethnicity, filling with mean.
Unseen value in column: jundice, filling with mean.
Unseen value in column: contry_of_res, filling with mean.
Unseen value in column: used_app_before, filling with mean.
Unseen value in column: relation, filling with mean.


<ipython-input-17-45f46bb9dbc2>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_df[col].fillna(t_int_encodings[col].mean(), inplace=True)
<ipython-input-17-45f46bb9dbc2>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

In [ ]:
# # Remove 'result' column as it's not used for prediction
# new_df = new_df.drop(columns=['result numeric'])

# Check for any remaining NaNs before prediction
if new_df.isna().any().any():
    print("Still has NaNs, please check input carefully.")
else:
    X_new_scaled = scaler.transform(new_df)
    prediction = model.predict(X_new_scaled)
    predicted_class = int(prediction[0][0] > 0.5)

    print("Predicted ASD Class:", "ASD" if predicted_class == 1 else "No ASD")
    print("Probability Score:", float(prediction[0][0]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted ASD Class: ASD
Probability Score: 0.9984903335571289
